#**Reading the data**

In [1]:
!pip install ar-corrector

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53.4 MB 27 kB/s 


In [2]:
import re
import pandas as pd
from collections import Counter
import nltk
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import numpy as np
from nltk.corpus import stopwords
import os
import warnings
import itertools
# import emoji
from nltk.corpus import stopwords
import string
from ar_corrector.corrector import Corrector

In [4]:
data = pd.read_csv("/content/MADAR_Cairo.csv")

In [5]:
teeext=data.iloc[11,0]
teeext

18

In [6]:
data

,sentID.BTEC,split,lang,sent
0,0,corpus-6-train,CAI,هو في اخر القاعة . أنا حأجيبلك شويه دلوقتي . ل...
1,2,corpus-6-train,CAI,بتعمل تبديلات؟
2,4,corpus-6-train,CAI,عايزين ترابيزة جنب الشباك .
3,5,corpus-6-test-corpus-26-train,CAI,ده قدامك هناك، يادوبك قدام مكتب استعلامات السي...
4,9,corpus-6-test-corpus-26-train,CAI,عمري ما سمعت عن العنوان ده هنا .
...,...,...,...,...
11995,19959,corpus-6-test-corpus-26-train,CAI,في حمام في المحطة.
11996,19960,corpus-6-train,CAI,سيبه طويل.
11997,19965,corpus-6-train,CAI,أنا خلصت البنزين.
11998,19968,corpus-6-train,CAI,ممكن آخد أي كرسيين متاحين لليلة دي؟


In [7]:
data['sent'].describe()

count                   12000
unique                  11912
top       فين مكتب المفقودات؟
freq                        4
Name: sent, dtype: object

In [8]:
data['lang'].unique()

array(['CAI'], dtype=object)

#**Preproccessing**

#### Data Cleaning & Normlization
>*  Remove Punctuatuions
*  Remove Usernames(mentions)
*   Remove URLs
*   Remove emoji
*   Remove Numbers
*   Remove Special Characters
*   Remove Elongation







In [9]:
!pip install farasapy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
!git clone https://github.com/aub-mind/arabert

Cloning into 'arabert'...
remote: Enumerating objects: 600, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 600 (delta 38), reused 45 (delta 30), pack-reused 535
Receiving objects: 100% (600/600), 9.14 MiB | 24.70 MiB/s, done.
Resolving deltas: 100% (339/339), done.


In [11]:
!pip install pyarabic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 126 kB 13.0 MB/s 


In [12]:
!pip install pyspellchecker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.5 MB 12.6 MB/s 


In [13]:
def removeshortsen(txt):
  words=txt.split(' ')
  text=''
  if len(words) > 4:
    text = ' '.join(words)

  return text

In [14]:
def removeMention(txt):
  arabic_men=re.sub("@[أ-ي]+","",txt)#//remove Arabic hashtags that have  after it
  arabic_men=re.sub("[أ-ي]@+","",arabic_men) #remove Arabic hashtags that have # before it
  without_men=re.sub("@[A-Za-z]+","",arabic_men)# remove English hashtags
  return without_men

In [15]:
def removeduplicate(txt):
  non_duplicate = ''.join(i for i, _ in itertools.groupby(txt))
  return non_duplicate


In [16]:
def reduce_characters(inputText):
    '''
    step #4: Reduce character repitation of > 2 characters at time
              For example: the word 'cooooool' will convert to 'cool'
    '''
    # pattern to look for three or more repetitions of any character, including
    # newlines.
    pattern = re.compile(r"(.)\1{2,}", re.DOTALL)
    reduced_text = pattern.sub(r"\1\1", inputText)
    return reduced_text

In [17]:
def removeDigits(txt):
  without_digit = re.sub('[0-9]+', '', txt) 
  return without_digit

In [18]:
def removeURl(txt):
  without_url= re.sub('((www\.[^\s]+)|(https?://[^\s]+))', ' ', txt)
  return without_url

In [19]:
def removestopwords(txt):
  without_stopwords = ' '.join(word for word in txt.split() if word not in stop_words)
  return without_stopwords

In [20]:
def removepuncatution(txt):

  punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ''' + string.punctuation
  translator = str.maketrans('', '', punctuations)
  without_punc = txt.translate(translator)

  # txtt = re.sub("[\s+\\# ـ!\/_,$%=^*?:@&^~`(+\"]+|[+！，。？、~@￥%……&*（）“ ”:;：；、\\《）《》“”()»〔〕# ]+ ·.『]", "", txt)
  # seq = re.sub("[\s+[`÷×؛<>_()*&% ـ،/:{} ÷×؛<>_()*&^%][ـ،/: ]", "", txtt)
  return without_punc

In [21]:
def removenonrabic(txt):
  # Remove None arabic 
  without_english = re.sub(u"[^\u0621-\u063A\u0640-\u0652 ]", " ", txt)
  return without_english

In [22]:
def removeHashtages(txt):
  arabic_hash=re.sub("#[أ-ي]+","",txt)#//remove Arabic hashtags that have  after it
  arabic_hash_=re.sub("[أ-ي]#+","",arabic_hash) #remove Arabic hashtags that have # before it
  without_hash=re.sub("#[A-Za-z]+","",arabic_hash)# remove English hashtags
  return without_hash

In [23]:
def removetashkeel(txt):
  TATWEEL = u"\u0640"
  p_tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
  without_tashkeel = re.sub(p_tashkeel,"", txt)
  without_tashkeel=without_tashkeel.replace(TATWEEL, '')
  return without_tashkeel

In [24]:
def remove_Emojis(text):
  emoj = re.compile("["
      u"\U0001F600-\U0001F64F"  # emoticons
      u"\U0001F300-\U0001F5FF"  # symbols & pictographs
      u"\U0001F680-\U0001F6FF"  # transport & map symbols
      u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
      u"\U00002500-\U00002BEF"  # chinese char
      u"\U00002702-\U000027B0"
      u"\U00002702-\U000027B0"
      u"\U000024C2-\U0001F251"
      u"\U0001f926-\U0001f937"
      u"\U00010000-\U0010ffff"
      u"\u2640-\u2642" 
      u"\u2600-\u2B55"
      u"\u200d"
      u"\u23cf"
      u"\u23e9"
      u"\u231a"
      u"\ufe0f"  # dingbats
      u"\u3030"
                    "]+", re.UNICODE)
  without_emojy=re.sub(emoj, '', text)
  return without_emojy

In [25]:
# corr = Corrector()
# def correction(text):
#   text=corr.contextual_correct(text)
#   return text

In [26]:
def Clean_Egyptain_text(raw_text):

  raw_text=raw_text.strip()

  #1 remove shorten sentences that contain less than 4 words 
  text=removeshortsen(raw_text)

  #2 remove duplichated chars in the raw text
  text=removeduplicate(text)
  #text=reduce_characters(text)

  #3 remove hashtages after remove duplicated chars
  text=removeHashtages(text)

  #4 remove digits
  text= removeDigits(text)

  #5 remove the puncatution from the text 
  text=removepuncatution(text)

  #6 remove the mention and @ char
  text=removeMention(text)

  #7 remove arabic tashkeel from the text
  text=removetashkeel(text)

  #8 remove Emojies from the text
  text=remove_Emojis(text)

  #9 remove url and links from the text
  text=removeURl(text)

  # #9 remove stopwords from the text
  # text=removestopwords(text)

  #10 remove non rabic words from the text
  text=removenonrabic(text)

  return text

In [27]:
# def Clean_Egyptain_text(raw_text):

#   raw_text=raw_text.strip()

#   #1 remove duplichated chars in the raw text
#   #text=removeduplicate(raw_text)
#   text=reduce_characters(raw_text)

#   #2 remove hashtages after remove duplicated chars
#   text=removeHashtages(text)

#   #3 remove digits
#   text= removeDigits(text)

#   #4 remove the puncatution from the text 
#   text=removepuncatution(text)

#   #5 remove the mention and @ char
#   text=removeMention(text)

#   #6 remove arabic tashkeel from the text
#   text=removetashkeel(text)

#   #7 remove Emojies from the text
#   text=remove_Emojis(text)

#   #8 remove url and links from the text
#   text=removeURl(text)

#   # #9 remove stopwords from the text
#   # text=removestopwords(text)

#   #10 remove non rabic words from the text
#   text=removenonrabic(text)

  

#   return text

In [28]:
clean = {
    'Text': data['sent'].apply(lambda x: Clean_Egyptain_text(str(x)))
    }
Cleaned_data = pd.DataFrame.from_dict(clean)

In [29]:
filter = Cleaned_data['Text'] != ""
Cleaned_data = Cleaned_data[filter]

In [30]:
Cleaned_data

,Text
0,هو في اخر القاعة أنا حأجيبلك شويه دلوقتي لو ...
2,عايزين ترابيزة جنب الشباك
3,ده قدامك هناك يادوبك قدام مكتب استعلامات السياحة
4,عمري ما سمعت عن العنوان ده هنا
5,أمشي علطول لحد ما تلقي صيدلية
...,...
11988,لو سمحت اصرف شيكات سياحية
11990,لأن في أمريكا من السهل إنك توصل لكلية بس صعب إ...
11993,إيه هي المعالم الي على الطريق
11994,أيوة أنا أنا رايح لوس أنجيلوس


#### Check missing data

In [31]:
Cleaned_data.isnull().sum()

Text    0
dtype: int64

#### Cleaned data CSV

In [32]:
Cleaned_data.to_csv('clean_dataset2.csv',encoding="utf8")

####Get the Vocab from thr trained data

In [33]:
x = Cleaned_data.to_string(header=False,
                  index=False,
                  index_names=False).split('\n')
words = [','.join(ele.split()) for ele in x]
print(words)

['هو,في,اخر,القاعة,أنا,حأجيبلك,شويه,دلوقتي,لو,محتاج,حاجة,تانية,قولي', 'عايزين,ترابيزة,جنب,الشباك', 'ده,قدامك,هناك,يادوبك,قدام,مكتب,استعلامات,السياحة', 'عمري,ما,سمعت,عن,العنوان,ده,هنا', 'أمشي,علطول,لحد,ما,تلقي,صيدلية', 'ايه,اجد,لون,في,الموسم,ده', 'في,الحالة,بتاعتي,بيبقى,عادة,شغل,وقليل,جدا,ترفيه', 'انا,عايز,اعمل,كيرلي,جامد', 'عايز,احجز,اوضتين,بسريرين', 'الأوتوبيس,ده,حيقف,في,شارع,ستونر', 'من,فضلك,افتح,بقك,جامد', 'احود,شمال,عند,الناصية,التالته', 'انا,اديتك,تليفونه,و,عنوانه', 'انا,دفعت,مقدما,حساب,الفندق,و,الاكل', 'بأدور,على,حاتي,كويس,ويكون,هادي,مكن,توريني,مكانهم,عالخريطة,دي', 'حأبلغها,تكلمك,اول,ما,ترجع', 'تحب,كريمة,و,سكر,في,قهوتك', 'فيه,في,الفندق,ده,تجهيزات,لمؤتمرات', 'حيكون,كويس,لو,اخدنا,وجبة,خفيفة', 'انا,بأكلمك,من,المطار,حأكون,هناك,حوالي,اربعه', 'من,فين,أقدر,أروح,لشيكاجو', 'بكام,أقدر,ابعت,جواب,مسجل,لفرنسا', 'مكن,تصرفلي,شيك,بميتين,دولار', 'ده,فعلا,مختلف,ان,الواحد,يسوق,على,جنب,الشارع,اليمين', 'فين,قسم,الكتب,الي,مش,خياليه,بتاعك', 'لو,لازم,يظهر,لو,سمحت,كلمني', 'مكن,لو,سمحت,تغير,الكرسي,بتاعي',

In [34]:
All_words = [word for item in x for word in item.split()]
All_words

['هو',
 'في',
 'اخر',
 'القاعة',
 'أنا',
 'حأجيبلك',
 'شويه',
 'دلوقتي',
 'لو',
 'محتاج',
 'حاجة',
 'تانية',
 'قولي',
 'عايزين',
 'ترابيزة',
 'جنب',
 'الشباك',
 'ده',
 'قدامك',
 'هناك',
 'يادوبك',
 'قدام',
 'مكتب',
 'استعلامات',
 'السياحة',
 'عمري',
 'ما',
 'سمعت',
 'عن',
 'العنوان',
 'ده',
 'هنا',
 'أمشي',
 'علطول',
 'لحد',
 'ما',
 'تلقي',
 'صيدلية',
 'ايه',
 'اجد',
 'لون',
 'في',
 'الموسم',
 'ده',
 'في',
 'الحالة',
 'بتاعتي',
 'بيبقى',
 'عادة',
 'شغل',
 'وقليل',
 'جدا',
 'ترفيه',
 'انا',
 'عايز',
 'اعمل',
 'كيرلي',
 'جامد',
 'عايز',
 'احجز',
 'اوضتين',
 'بسريرين',
 'الأوتوبيس',
 'ده',
 'حيقف',
 'في',
 'شارع',
 'ستونر',
 'من',
 'فضلك',
 'افتح',
 'بقك',
 'جامد',
 'احود',
 'شمال',
 'عند',
 'الناصية',
 'التالته',
 'انا',
 'اديتك',
 'تليفونه',
 'و',
 'عنوانه',
 'انا',
 'دفعت',
 'مقدما',
 'حساب',
 'الفندق',
 'و',
 'الاكل',
 'بأدور',
 'على',
 'حاتي',
 'كويس',
 'ويكون',
 'هادي',
 'مكن',
 'توريني',
 'مكانهم',
 'عالخريطة',
 'دي',
 'حأبلغها',
 'تكلمك',
 'اول',
 'ما',
 'ترجع',
 'تحب',
 'كريمة',


In [35]:
type(All_words)

list

In [36]:
# file.write("\n".join(names)

In [37]:
with open("file.txt", "w") as output:
    output.write(str(All_words))